In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import joblib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [ ]:
# Paths
DATA_CSV = "lstm_hourly_data_p_s2.csv"     
MODEL_DIR = "lstm_models_ps2"
SCALER_DIR = "lstm_scalers_ps2"
PLOTS_DIR = "lstm_plots_ps2"

# Create directories
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(SCALER_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)

# Training settings
N_STEPS = 24
EPOCHS = 60
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.1
PATIENCE = 8
LAST_N_HOURS_TO_PLOT = 200


In [ ]:
def create_sequences(values, n_steps=24):
    X, y = [], []
    for i in range(len(values) - n_steps):
        X.append(values[i:i+n_steps])
        y.append(values[i+n_steps])
    return np.array(X), np.array(y).reshape(-1, 1)

def build_model(n_steps):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(n_steps, 1)),
        Dropout(0.1),
        LSTM(32),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [ ]:
df = pd.read_csv(DATA_CSV, parse_dates=['localminute'])
df = df.sort_values('localminute').reset_index(drop=True)

house_ids = df['dataid'].unique()
print("Total houses found:", len(house_ids))
house_ids
